# Intersight Python SDK Installation

If you are familiar or proficient with Python, you can take advantage of the Intersight Python SDK available for install from the [Python Package Index](https://pypi.org/project/intersight/).

![Python SDK for Cisco Intersight](images/python_sdk_pip.png)

Notice the `pip install intersight` command. That's the command used to install the Python SDK in your environment.  Be sure you are using Python >= 3.6, earlier versions of Python are not supported.


In [24]:
pip install intersight

You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.



> Be sure to uninstall any conflicting versions of the SDK if you have previous installs.  You can check installed versions with pip list.

In [25]:
pip list

Package             Version
------------------- ----------
ansible             4.2.0
ansible-core        2.11.2
anyio               3.3.0
appnope             0.1.2
argon2-cffi         20.1.0
arrow               1.1.1
attrs               21.2.0
autopep8            1.5.7
Babel               2.9.1
backcall            0.2.0
binaryornot         0.4.4
bleach              4.0.0
certifi             2021.5.30
cffi                1.14.5
chardet             4.0.0
charset-normalizer  2.0.4
click               8.0.1
cookiecutter        1.7.3
cryptography        3.4.7
debugpy             1.4.1
decorator           5.0.9
defusedxml          0.7.1
entrypoints         0.3
idna                3.2
imcsdk              0.9.10
intersight          1.0.9.4437
ipykernel           6.2.0
ipython             7.26.0
ipython-genutils    0.2.0
ipywidgets          7.6.3
jedi                0.18.0
Jinja2              3.0.1
jinja2-time         0.2.0
json5               0.9.6
jsonschema          3.2.0
jupyter            

> If you see Intersight-OpenAPI installed, you can run `pip unistall Intersight-OpenAPI`

## API Keys and User Authentication

Now that the Intersight Python SDK is installed, let's use it to connect to our Intersight environment.  First, you'll need an Intersight API Key ID and secret (private) key from your Intersight account.  From the Settings menu in Intersight, Select API Keys and Generate API Key

![Generate API Key](images/generate_api_key.png)

A version 3 key can be used with the SDK and is recommended for long term compatibility with Intersight's API.

![Version 3 key](images/version_3_key.png)

The Generated API Key ID can be copied to the clipboard and used in API authentication below.  Also be sure to save the Secret (private) Key to a local file that only you have access to (a download option is provided by Intersight).

![Download Secret Key](images/secret_key_download.png)

### Using the Intersight Python SDK

To use the SDK, you must import required modules.  You can view help once you've imported any required modules.


In [26]:
import intersight

# View the help
help(intersight)

Help on package intersight:

NAME
    intersight - Cisco Intersight

DESCRIPTION
    Cisco Intersight is a management platform delivered as a service with embedded analytics for your Cisco and 3rd party IT infrastructure. This platform offers an intelligent level of management that enables IT organizations to analyze, simplify, and automate their environments in more advanced ways than the prior generations of tools. Cisco Intersight provides an integrated and intuitive management experience for resources in the traditional data center as well as at the edge. With flexible deployment options to address complex security needs, getting started with Intersight is quick and easy. Cisco Intersight has deep integration with Cisco UCS and HyperFlex systems allowing for remote deployment, configuration, and ongoing maintenance. The model-based deployment works for a single system in a remote location or hundreds of systems in a data center and enables rapid, standardized configuration and depl

### API Client Configuration

To use the SDK, you will need to configure an API client using the API Keys you created above.  You can view the help for Intesight's Configuration and ApiClient classes to see what is needed.

In [27]:
help(intersight.Configuration)


Help on class Configuration in module intersight.configuration:

class Configuration(builtins.object)
 |  Configuration(host=None, api_key=None, api_key_prefix=None, access_token=None, username=None, password=None, discard_unknown_keys=False, disabled_client_side_validations='', signing_info=None, server_index=None, server_variables=None, server_operation_index=None, server_operation_variables=None, ssl_ca_cert=None)
 |  
 |  NOTE: This class is auto generated by OpenAPI Generator
 |  
 |      Ref: https://openapi-generator.tech
 |      Do not edit the class manually.
 |  
 |      :param host: Base url
 |      :param api_key: Dict to store API key(s).
 |        Each entry in the dict specifies an API key.
 |        The dict key is the name of the security scheme in the OAS specification.
 |        The dict value is the API key secret.
 |      :param api_key_prefix: Dict to store API prefix (e.g. Bearer)
 |        The dict key is the name of the security scheme in the OAS specification.

In [28]:
help(intersight.ApiClient)

Help on class ApiClient in module intersight.api_client:

class ApiClient(builtins.object)
 |  ApiClient(configuration=None, header_name=None, header_value=None, cookie=None, pool_threads=1)
 |  
 |  Generic API client for OpenAPI client library builds.
 |  
 |  OpenAPI generic API client. This client handles the client-
 |  server communication, and is invariant across implementations. Specifics of
 |  the methods and models for each application are generated from the OpenAPI
 |  templates.
 |  
 |  NOTE: This class is auto generated by OpenAPI Generator.
 |  Ref: https://openapi-generator.tech
 |  Do not edit the class manually.
 |  
 |  :param configuration: .Configuration object for this client
 |  :param header_name: a header to pass when making calls to the API.
 |  :param header_value: a header value to pass when making calls to
 |      the API.
 |  :param cookie: a cookie to include in the header when making calls
 |      to the API
 |  :param pool_threads: The number of thread

### Example Usage

We're now ready to create an API client and use it to connect with Intersight.

> The values below only serve as an example so they won't work in your environment.  Be sure to enter the values for your account in the `key_id` variable below as well as the `private_key_path`.  Also be sure that you are using v3 API keys from your Intersight account.

In [54]:
configuration = intersight.Configuration(
    signing_info=intersight.HttpSigningConfiguration(
        key_id='596cc79e5d91b400010d15ad/5db71f977564612d30cc3860/5e9217a57564612d302f475b',
        private_key_path='/Users/dsoper/Downloads/SecretKey.txt',
        signing_scheme=intersight.signing.SCHEME_HS2019,
        signed_headers=[intersight.signing.HEADER_HOST,
                        intersight.signing.HEADER_DATE,
                        intersight.signing.HEADER_DIGEST,
                        intersight.signing.HEADER_REQUEST_TARGET
                        ]
    )
)
api_client = intersight.ApiClient(configuration)


#### Querying the API

Now that you are authenticated, let's use the SDK to query the Alarms resource in Intersight.  The example below sets up a query filter to get any Critical alarms that have occurred in the last 7 days.

In [67]:
from datetime import datetime, timedelta
import intersight.api.cond_api

# Get condition class instance
api_instance = intersight.api.cond_api.CondApi(api_client)

# Find all Critical severity alarms within the last 7 days
search_period = datetime.now() - timedelta(days=7)
# Convert date object to formatted date string for use with intersight queries
date_str = f"{search_period.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3]}Z"
query_filter = f"Severity eq Critical and CreationTime gt {date_str}"
# Only include CreationTime, Description, Code, and Acknowledge in results
query_select = "CreationTime,Description,Code,Acknowledge"

# Get alarms using query parameters
alarm_query = api_instance.get_cond_alarm_list(filter=query_filter, select=query_select)

print(alarm_query)


{'object_type': 'cond.Alarm.List',
 'results': [{'acknowledge': 'Acknowledge',
              'class_id': 'cond.Alarm',
              'code': 'EquipmentIoCardOffline',
              'creation_time': datetime.datetime(2021, 9, 17, 18, 28, 12, 364000, tzinfo=tzutc()),
              'description': 'IO Module B26-Matt-FI/chassis-1/iocard-2 is '
                             'offline',
              'moid': '6144de3c65696e2d30e820a7',
              'object_type': 'cond.Alarm'}]}


### Updating Resources

Using data from the query above, we can update Alarm resources to acknowledge alarms.  To acknowledge an Alarm, set the acknowledge attribute to `Acknowledge`.

In [66]:
import intersight.model.cond_alarm

alarm = intersight.model.cond_alarm.CondAlarm(acknowledge='Acknowledge')
alarm_patch = api_instance.patch_cond_alarm(moid=alarm_query.results[0].moid, cond_alarm=alarm)

You can re-run the query example to confirm that the patch operation has changed Acknowledge state on the alarm.

### Additional Examples

Several additional examples of using the SDK are on GitHub in the [Intersight Python Utilties repo](https://github.com/CiscoDevNet/intersight-python-utils).  The repo also contains a credentials.py module to simplify authentication across all of the example scripts.

### How to Learn More

Additional information on the Intersight API is available on the [Intersight API Docs Pages](https://www.intersight.com/apidocs).  Intersight also features an in system [API schema browser](https://www.intersight.com/apidocs/apirefs/) where developers can view the complete resource model and interact with the API in their account.  As an example, the schema for Alarms can be viewed at https://www.intersight.com/apidocs/apirefs/aaa/AuditRecords/model/ and if you are signed in to your Intersight account you can also perform operations against the API.

![API Reference Alarms](images/api_ref_alarms.png)

Your browser developer console can also be used to view API calls (all browser actions use the API).  In Chrome for example, you can right click and select Inspect to view the developer console.

![Chrome Inspect](images/chrome_inspect.png)

The Network tab shows details on all API calls performed by the browser including query strings used to retrieve specific data.

![Chrome Network Tab](images/chrome_network_tab.png)